### Import and Load data

In [1]:
from numpy import loadtxt
import numpy as np
from xgboost import XGBClassifier
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

In [2]:
# one hot encoding
def onehot(y):
    label = []
    for i in y:
        label.append([int(i)])
    y_result = to_categorical(label).astype(int)
    
    return y_result

In [3]:
# load data
train = loadtxt('../data/csvs/train_data_13pose.csv', delimiter=",")
test = loadtxt('../data/csvs/test_data_13pose.csv', delimiter=",")

In [4]:
df = np.concatenate((train, test))
X = df[:, :-1]
y = df[:, -1]

In [5]:
data_dmatrix = xgb.DMatrix(data=X, label=y)

In [6]:
X_train = train[:, :-1]
y_train_split = train[:, -1]
X_test = test[:, :-1]
y_test_split = test[:, -1]

In [7]:
y_train = onehot(y_train_split)
y_test = onehot(y_test_split)

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1199, 22)
(293, 22)
(1199, 13)
(293, 13)


In [9]:
# fit model no training data
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state=32)
model.fit(X_train, y_train)
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.2, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=4, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=32, ...)


In [10]:
expected_y = y_test
predicted_y = model.predict(X_test)

In [15]:
predicted_y[0]

array([1., 0., 0.])

In [11]:
accuracy = accuracy_score(expected_y, predicted_y)

In [12]:
print("Accuracy: %.2f%%" % (accuracy * 100))

Accuracy: 87.71%


In [13]:
from xgboost import cv

params = {"colsample_bytree":0.3, "learning_rate": 0.1, "max_depth": 5, "alpha": 10}

xgb_cv = cv(dtrain=data_dmatrix, params=params, nfold=3, num_boost_round=50, early_stopping_rounds=10, metrics="rmse", as_pandas=True)

In [14]:
xgb_cv.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,5.973750,0.023389,5.985690,0.044318
1,5.520471,0.019453,5.546610,0.053055
2,5.122560,0.015130,5.167537,0.063330
3,4.772867,0.016747,4.839407,0.055561
4,4.468242,0.023462,4.551089,0.051684


In [15]:
actions = np.array(['hands_up', 'neck_down', 'neck_side'])
actions[np.argmax(predicted_y[0])]

'hands_up'

### Save Weights

In [16]:
model.save_model("xgb_13pose.json")